In [463]:
import pandas as pd
import numpy as np
ref = pd.read_csv("FTIR_classification.csv")
d3 = pd.read_csv("Yan2023Data.csv")
print(ref.shape)
print(d3.shape)

(325, 1876)
(959, 1765)


Interpolation of the new data set to merge with the main data

In [464]:
freq_candidates = ref.columns[1:-11]
clean_freqs = []
clean_names = []
for col in freq_candidates:
  c = col.replace(",", ".").strip()
  filtered = ''.join(ch for ch in c if (ch.isdigit() or ch == "."))
  if filtered == "":
    continue
  try:
    freq_val = float(filtered)
    clean_freqs.append(freq_val)
    clean_names.append(col)
  except ValueError:
    pass
clean_freqs = np.array(clean_freqs)
clean_names = np.array(clean_names)
print("Clean freq#:", len(clean_freqs))
print("Example:", clean_freqs[:10])

Clean freq#: 1864
Example: [3992.6 3990.7 3988.8 3986.8 3984.9 3983.  3981.1 3979.1 3977.2 3975.3]


In [465]:
import numpy as np
target_freqs = np.sort(clean_freqs) #increasing order
print("Target (referans) freq range:",target_freqs[0], "→", target_freqs[-1]," | #", len(target_freqs))
freq_cols_d3 = d3.columns[2:-1]
source_freqs = np.array([float(c) for c in freq_cols_d3])
source_freqs_sorted = np.sort(source_freqs)
print("Yan2023Data freq range:",source_freqs_sorted[0], "→", source_freqs_sorted[-1]," | #:", len(source_freqs_sorted))


Target (referans) freq range: 401.2 → 3992.6  | # 1864
Yan2023Data freq range: 599.78354 → 3995.98555  | #: 1762


focus on the intersection frequencies we do not know the absorbence of the at the frequecies lower than the 599/7854

In [466]:
import numpy as np
order_ref = np.argsort(clean_freqs)
target_freqs = clean_freqs[order_ref]
target_names = np.array(clean_names)[order_ref]
print("Target freq range:", target_freqs[0], "→", target_freqs[-1])
print("Target count:", len(target_freqs))

Target freq range: 401.2 → 3992.6
Target count: 1864


In [467]:
freq_cols_d3 = d3.columns[2:-1]
source_freqs = np.array([float(c) for c in freq_cols_d3])
order_src = np.argsort(source_freqs)
source_freqs_sorted = source_freqs[order_src]
freq_cols_d3_sorted = freq_cols_d3[order_src]
print("newData freq range:", source_freqs_sorted[0], "→", source_freqs_sorted[-1])
print("newData freq count:", len(source_freqs_sorted))
low = max(target_freqs[0],  source_freqs_sorted[0])
high = min(target_freqs[-1], source_freqs_sorted[-1])
mask_target = (target_freqs >= low) & (target_freqs <= high)
target_freqs_overlap = target_freqs[mask_target]
print("Overlap freq range:", low, "→", high)
print("Overlap count:", len(target_freqs_overlap))

newData freq range: 599.78354 → 3995.98555
newData freq count: 1762
Overlap freq range: 599.78354 → 3992.6
Overlap count: 1761


In [468]:
X_aligned = []
labels_name = []
labels_id = []

for _, row in d3.iterrows():
    y_src = row[freq_cols_d3_sorted].to_numpy(dtype=float)
    y_interp = np.interp(target_freqs_overlap, source_freqs_sorted, y_src)

    X_aligned.append(y_interp)
    labels_name.append(row["Interpretation "])
    labels_id.append(row["polymerID"])

X_aligned = np.vstack(X_aligned)
print("Aligned X shape:", X_aligned.shape)

Aligned X shape: (959, 1761)


In [469]:
col_names = [f"{f:.4f}" for f in target_freqs_overlap]

df_d3_aligned = pd.DataFrame(X_aligned, columns=col_names)
df_d3_aligned["Interpretation"] = labels_name
df_d3_aligned["polymerID"] = labels_id

print(df_d3_aligned.shape)
df_d3_aligned.head()

(959, 1763)


,599.9000,601.8000,603.7000,605.6000,607.6000,609.5000,611.4000,613.4000,615.3000,617.2000,...,3979.1000,3981.1000,3983.0000,3984.9000,3986.8000,3988.8000,3990.7000,3992.6000,Interpretation,polymerID
0,0.647726,0.801826,1.195663,1.672671,1.868343,1.536074,1.117544,1.108418,1.138807,0.787730,...,0.074123,0.133818,0.183874,0.190147,0.153343,0.099131,0.051876,0.022910,Ethylene propylene rubber,1
1,0.876380,1.226408,0.893337,0.404535,0.434476,1.045745,1.719252,1.849145,1.563762,1.354734,...,0.040134,-0.002571,-0.068482,-0.084733,-0.017509,0.082759,0.130164,0.124266,Ethylene propylene rubber,1
2,0.495661,0.495621,0.578984,0.676564,0.633753,0.457439,0.446798,0.778201,1.082610,1.072515,...,0.029860,0.019271,0.033783,0.058673,0.066975,0.054406,0.033962,0.003428,Ethylene propylene rubber,1
3,0.702158,0.820559,0.767265,0.671184,0.598374,0.513017,0.506760,0.631545,0.744831,0.744863,...,0.025641,-0.007404,-0.018939,0.000677,0.020917,0.026424,0.022590,0.011754,Ethylene propylene rubber,1
4,0.633943,0.658256,0.799107,0.928757,0.824523,0.575465,0.412906,0.526334,0.761193,0.831783,...,0.048374,0.014350,0.024654,0.049763,0.044706,0.020827,0.012385,0.009680,Ethylene propylene rubber,1


interpolation in ascending order not descending. Interpretation will be changed with substance later and label column will be included.

In [470]:
df_d3_aligned.to_csv("newData_aligned_to_FTIR_classification.csv", index=False)

Now interpolation of FTIR Classification to merge both data set

In [471]:
import numpy as np
import pandas as pd

In [472]:
ref = pd.read_csv("FTIR_classification.csv")

In [473]:
ref = ref.loc[:, ~ref.columns.str.contains('^Unnamed')] # for unnamed column

In [474]:
freq_cols_ref = []
for col in ref.columns:
  name = col.replace(",", ".").strip()
  try:
      float(name)
      freq_cols_ref.append(col)
  except ValueError:
      pass

print("Column #:", len(freq_cols_ref))

ref_freqs = np.array([float(c.replace(",", ".").strip()) for c in freq_cols_ref])

Column #: 1863


Delete meta data from main data and convert the absorbence values to float

In [475]:
order = np.argsort(ref_freqs) # ascending order
ref_freqs_sorted = ref_freqs[order]

In [476]:
aligned_rows = []
for _, row in ref.iterrows():
    y_src = row[freq_cols_ref].astype(str).str.replace(",", ".").astype(float).to_numpy()
    y_src_sorted = y_src[order]
    y_interp = np.interp(target_freqs_overlap, ref_freqs_sorted, y_src_sorted)
    aligned_rows.append(y_interp)
col_names = [f"{f:.4f}" for f in target_freqs_overlap]
df_ftir_aligned = pd.DataFrame(aligned_rows, columns=col_names)

Adding the other columns again we use it for plastin non plastic approach

In [477]:
meta_cols = ["ID", "Substance", "Natural /Synthetic", "Plastic/other",
             "framework", "Abbreviation", "Source", "Source ID",
             "Form", "Colour", "Method"]

df_ftir_aligned = pd.concat([ref[meta_cols].reset_index(drop=True),
                             df_ftir_aligned], axis=1)

In [478]:
print(df_ftir_aligned.shape)
df_ftir_aligned.head()

(325, 1772)


,ID,Substance,Natural /Synthetic,Plastic/other,framework,Abbreviation,Source,Source ID,Form,Colour,...,3975.3000,3977.2000,3979.1000,3981.1000,3983.0000,3984.9000,3986.8000,3988.8000,3990.7000,3992.6000
0,1,1_2_polybutadiene,synthetic polymer,plastic,polybutadiene,1_2_polybutadiene,"Scientific Polymer Products, Inc.",688,beads,transparent,...,0.00050,0.00055,0.00059,0.00063,0.00068,0.00071,0.00074,0.00080,0.00087,0.00095
1,2,acrylonitrile_butadiene_styrene,synthetic polymer,plastic,acrylonitrile butadiene styrene,ABS,RSH Polymere,NaN,foil,light brown,...,0.00059,0.00060,0.00065,0.00073,0.00078,0.00083,0.00090,0.00094,0.00098,0.00100
2,3,acrylonitrile_butadiene_styrene,synthetic polymer,plastic,acrylonitrile butadiene styrene,ABS,RSH Polymere,NaN,foil,light brown,...,0.00046,0.00052,0.00060,0.00066,0.00069,0.00072,0.00073,0.00073,0.00074,0.00078
3,4,fibre_acetate,synthetic polymer,modified cellulose,cellulose acetate,fibre_acetate,Faserinstitut Bremen,NaN,fibre,white,...,0.00023,0.00026,0.00026,0.00025,0.00027,0.00030,0.00029,0.00028,0.00029,0.00031
4,5,acrylonitrile_butadiene,synthetic polymer,plastic,acrylonitrile butadiene,acrylonitrile_butadiene,"Scientific Polymer Products, Inc.",530,crumbs,yellow,...,0.00015,0.00016,0.00018,0.00019,0.00022,0.00022,0.00023,0.00027,0.00030,0.00031


In [479]:
df_ftir_aligned.to_csv("FTIR_classification_aligned.csv", index=False)

The main data aligned. The main issue is adding label to the additional data set and change the column name according to the main data

In [480]:
df_d3_aligned = df_d3_aligned.rename(columns={"polymerID": "ID"})
df_d3_aligned = df_d3_aligned.rename(columns={"Interpretation": "Substance"})

add meta data

In [481]:
for col in ["Natural /Synthetic", "Plastic/other", "framework","Abbreviation", "Source", "Source ID","Form", "Colour", "Method"]:
   df_d3_aligned[col] = None

add plastic non plastic column but not labeled yet

In [482]:
df_d3_aligned = df_d3_aligned.rename(columns=lambda c: c.strip())
df_d3_aligned["is_plastic"] = (df_d3_aligned["Plastic/other"] == "plastic").astype(int)
print(df_d3_aligned["is_plastic"].value_counts())

is_plastic
0    959
Name: count, dtype: int64


In [483]:
freq_cols = [c for c in df_d3_aligned.columns if c.replace('.', '').isdigit()]
df_d3_aligned = df_d3_aligned[["ID", "Substance","Natural /Synthetic", "Plastic/other", "framework","Abbreviation", "Source", "Source ID","Form", "Colour", "Method"]+ freq_cols+ ["is_plastic"]]

In [484]:
df_d3_aligned.to_csv("new_data_aligned_for_merge.csv", index=False)

In [485]:
df_d3_aligned["Substance"].value_counts().head(30)

,count
Substance,
Poly(propylene),243
Poly(ethylene),228
Poly(ethylene) + fouling,148
Cellulose acetate,70
Poly(styrene),66
Morphotype 1,55
Ethylene propylene rubber,49
PEVA,43
Poly(ethylene) like,25


Non plastic: Cellulose acetate morphotype1 morphotype2 for yan2023Data

In [486]:
df_d3_aligned["is_plastic"] = df_d3_aligned["Substance"].str.lower().apply(lambda x: 0 if ("cellulose" in x or "morphotype 1" in x or "morphotype 2" in x) else 1)


In [487]:
df_d3_aligned["is_plastic"].value_counts()

,count
is_plastic,
1,823
0,136


In [488]:
df_d3_aligned["Plastic/other"] = df_d3_aligned["is_plastic"].map({1: "plastic", 0: "other"})

In [489]:
is_plastic_col = df_d3_aligned.pop("is_plastic")
pos = df_d3_aligned.columns.get_loc("Plastic/other") + 1
df_d3_aligned.insert(pos, "is_plastic", is_plastic_col)

In [490]:
df_d3_aligned.to_csv("new_data_aligned_final_for_merge.csv", index=False)

In [491]:
merged_df = pd.concat([df_ftir_aligned, df_d3_aligned], ignore_index=True)

In [492]:
merged_df.to_csv("merged_FTIR_and_Yan_dataset.csv", index=False)

the merged file contains is plastic coluiimn needs to be deelted and the ID numbre is not asccending

In [493]:
import pandas as pd
merged = pd.read_csv("merged_FTIR_and_Yan_dataset.csv")
merged = merged.iloc[:, :-1]
merged["orig_ID"] = merged["ID"]
merged["ID"] = range(1, len(merged) + 1)
merged.to_csv("merged_FTIR_and_Yan_dataset_clean.csv", index=False)

/tmp/ipython-input-410600792.py:2: DtypeWarning: Columns (2,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  merged = pd.read_csv("merged_FTIR_and_Yan_dataset.csv")


we will augment the data with different functions noise, scaling, wavenumber shift, mixup to prevent overfitting
Gaussian noise: adds small random fluctuations to the spectrum
Random scaling:multiplies the spectrum by a small random factor
Wavenumbershift: aplies a very small horizantal shift to the spectrum
Mixup:by blending two real spectra using a random mixing coefficient


In [494]:
def augment_noise(df, noise_level=0.05, n_samples=1): #started with 0.01 not changed a lot then changed to 0.5
  num_cols = df.select_dtypes(include=[np.number]).columns
  numeric_cols = [c for c in df.columns if c not in ["ID", "orig_ID", "is_plastic"]]
  augmented = []
  for _ in range(n_samples):
    noisy = df.copy()
    noisy[numeric_cols] = noisy[numeric_cols].apply(pd.to_numeric, errors='coerce')
    noise = np.random.normal(0, noise_level, size=noisy[numeric_cols].shape)
    noisy[numeric_cols] = noisy[numeric_cols] + noise
    augmented.append(noisy)
  return pd.concat(augmented, ignore_index=True)

In [495]:
def augment_scaling(df, scale_range=(0.95, 1.05), n_samples=1):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    augmented = []

    for _ in range(n_samples):
        scaled = df.copy()
        factor = np.random.uniform(scale_range[0], scale_range[1])
        scaled[numeric_cols] = scaled[numeric_cols] * factor
        augmented.append(scaled)

    return pd.concat(augmented, ignore_index=True)

In [496]:
def augment_shift(df, shift_pixels=1):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    shifted = df.copy()
    shifted[numeric_cols] = shifted[numeric_cols].shift(shift_pixels, axis=1)
    shifted = shifted.fillna(method='bfill', axis=1).fillna(method='ffill', axis=1)
    return shifted

In [497]:
def augment_mixup(df, alpha=0.5, n_samples=1):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    augmented = []

    for _ in range(n_samples):
        idx1 = np.random.randint(0, len(df))
        idx2 = np.random.randint(0, len(df))

        sample1 = df.iloc[idx1].copy()
        sample2 = df.iloc[idx2].copy()

        a = np.random.uniform(alpha, 1-alpha)
        mixed = sample1.copy()
        mixed[numeric_cols] = a * sample1[numeric_cols] + (1-a) * sample2[numeric_cols]

        augmented.append(mixed.to_frame().T)

    return pd.concat(augmented, ignore_index=True)

In [498]:
df = pd.read_csv("merged_FTIR_and_Yan_dataset_clean.csv")

/tmp/ipython-input-1936993126.py:1: DtypeWarning: Columns (2,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("merged_FTIR_and_Yan_dataset_clean.csv")


In [499]:
aug_noise = augment_noise(df, noise_level=0.05, n_samples=1)
aug_scale = augment_scaling(df, scale_range=(0.95, 1.05), n_samples=1)
aug_shift = augment_shift(df, shift_pixels=1)
aug_mix  = augment_mixup(df, alpha=0.5, n_samples=200)

/tmp/ipython-input-3118989833.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  shifted = shifted.fillna(method='bfill', axis=1).fillna(method='ffill', axis=1)
/tmp/ipython-input-3118989833.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  shifted = shifted.fillna(method='bfill', axis=1).fillna(method='ffill', axis=1)
/tmp/ipython-input-3118989833.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  shifted = shifted.fillna(method='bfill', axis=1).fillna(method='ffill', axis=1)


In [500]:
final_augmented = pd.concat([df, aug_noise, aug_scale, aug_shift, aug_mix],ignore_index=True)

In [501]:
final_augmented.to_csv("merged_FTIR_augmented_dataset.csv", index=False)

In [502]:
numeric_cols = df.select_dtypes(include=['float64','int64']).columns
diff = (aug_noise[numeric_cols] - df[numeric_cols]).abs().describe()
diff.head()

,ID,599.9000,601.8000,603.7000,605.6000,607.6000,609.5000,611.4000,613.4000,615.3000,...,3977.2000,3979.1000,3981.1000,3983.0000,3984.9000,3986.8000,3988.8000,3990.7000,3992.6000,orig_ID
count,1284.0,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,...,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1284.0
mean,0.0,0.039589,0.038450,0.039828,0.040012,0.039533,0.040135,0.040787,0.040629,0.040484,...,0.041494,0.039630,0.040616,0.041156,0.039743,0.040099,0.040752,0.041055,0.038553,0.0
std,0.0,0.028883,0.028879,0.030545,0.029758,0.030623,0.030865,0.029608,0.030305,0.029931,...,0.030230,0.029874,0.029969,0.030665,0.031165,0.030474,0.031139,0.031191,0.029106,0.0
min,0.0,0.000003,0.000135,0.000033,0.000015,0.000087,0.000003,0.000053,0.000028,0.000050,...,0.000027,0.000045,0.000007,0.000113,0.000012,0.000019,0.000044,0.000049,0.000039,0.0
25%,0.0,0.016448,0.014494,0.015229,0.016086,0.015966,0.015200,0.017464,0.016878,0.015787,...,0.017847,0.016140,0.015926,0.016205,0.015485,0.016124,0.015996,0.016280,0.015298,0.0
